In [21]:
import re
import torch
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
stemmer = WordNetLemmatizer()

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1123)>


In [3]:
""" load train feature dataframe """

train_df = pd.read_csv('perfect_final_train.csv')
train_df

,id,cell_id,cell_type,rank,source,fold
0,00001756c60be8,"['1862f0a6', '2a9e43d6', '038b763d', '2eefe0ef...","['code', 'code', 'code', 'code', 'code', 'code...","[0, 2, 4, 6, 8, 10, 14, 16, 18, 20, 22, 24, 25...",['# This Python 3 environment comes with many ...,0
1,00015c83e2717b,"['c417225b', '51e3cd89', '2600b4eb', '75b65993...","['code', 'code', 'code', 'code', 'code', 'code...","[4, 5, 6, 7, 8, 9, 12, 15, 16, 17, 18, 21, 22,...",['import numpy as np # linear algebra import p...,0
2,0001bdd4021779,"['3fdc37be', '073782ca', '8ea7263c', '80543cd8...","['code', 'code', 'code', 'code', 'code', 'code...","[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 8, 12]",['import pandas as pd import numpy as np impor...,1
3,0001daf4c2c76d,"['86605076', 'df6c939f', '00f87d0a', '986fd4f1...","['code', 'code', 'code', 'code', 'code', 'code...","[2, 5, 7, 11, 13, 14, 15, 17, 18, 19, 20, 21, ...",['# This Python 3 environment comes with many ...,1
4,0002115f48f982,"['18281c6c', 'e3b6b115', '4a044c54', '365fe576...","['code', 'code', 'code', 'code', 'code', 'code...","[1, 2, 3, 4, 5, 6, 7, 8, 0]",['import numpy as np # linear algebra import p...,2
...,...,...,...,...,...,...
139251,fffc30d5a0bc46,"['ff1ea6a0', 'a01ce9b3', 'bf92a015', '095812e6...","['code', 'code', 'code', 'code', 'code', 'code...","[1, 3, 5, 7, 9, 11, 12, 13, 15, 16, 17, 18, 19...","[""import warnings warnings filterwarnings 'ign...",4
139252,fffc3b44869198,"['978a5137', 'faa48f03', '28dfb12a', 'eea2e812...","['code', 'code', 'code', 'code', 'code', 'code...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14,...",['import matplotlib pyplot as plt import numpy...,0
139253,fffc63ff750064,"['5015c300', '8238198c', 'f4781d1d', 'b5532930...","['code', 'code', 'code', 'code', 'code', 'code...","[0, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 1...","[""import numpy as np import pandas as pd impor...",4
139254,fffcd063cda949,"['7e6266ad', 'd8281fc5', '3e0e4a47', '21387fc8...","['code', 'code', 'code', 'code', 'code', 'code...","[0, 1, 3, 4, 6, 7, 8, 10, 12, 14, 16, 17, 18, ...","[""import os GPU_id 0 os environ 'CUDA_VISIBLE_...",4


In [2]:
""" load full feature dataframe """

full_df = pd.read_csv('perfect_tmp_train.csv')
full_df

,id,cell_id,cell_type,source,rank,fold
0,00001756c60be8,1862f0a6,code,# This Python 3 environment comes with many he...,0,0
1,00001756c60be8,2a9e43d6,code,import numpy as np import pandas as pd import ...,2,0
2,00001756c60be8,038b763d,code,import warnings warnings filterwarnings 'ignore',4,0
3,00001756c60be8,2eefe0ef,code,matplotlib rcParams update 'font.size' 14,6,0
4,00001756c60be8,0beab1cd,code,def evaluate_preds train_true_values train_pre...,8,0
...,...,...,...,...,...,...
6370642,fffe1d764579d5,0d770d6b,markdown,## REMOVING THE OUTLIERS,43,4
6370643,fffe1d764579d5,d45ddc62,markdown,### DIMENSIONALITY CURSE,33,4
6370644,fffe1d764579d5,1a63248d,markdown,# BANGALORE HOUSE PRICE PREDICTION,0,4
6370645,fffe1d764579d5,a8ffc8b4,markdown,* We have achieved 75.2% accuracy in predictin...,69,4


In [27]:
""" Data Preprocessing Util Function """

def zero_filtering(x: torch.Tensor) -> torch.Tensor:
    """
    Add eps value for zero embedding, because competition metric is cosine similarity
    Cosine Similarity will be returned NaN, when input value has zero
    """
    eps = 1e-8
    x[x == 0] = eps
    return x

def create_word_normalizer():
    """
    Create a function that normalizes a word.
    """
    ps = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    def normalize(word):
        w = word.lower()
        w = lemmatizer.lemmatize(w)
        w = ps.stem(w)
        return w
    return normalize

def __normalize_words(titles: list) -> list:
    """
    Normalize a list of words
    1) Remove stop words
    2) Apply Porter Stemmer, Lemmatizer
    """
    stop_words = set(stopwords.words('english'))
    normalizer = create_word_normalizer()
    titles = [normalizer(t) for t in titles if t not in stop_words]
    return titles

def normalize_words(words: np.ndarray, unique=True) -> list:
    """
    Normalize a list of words
    1) Apply __normalize_word function
    2) Apply Regular Expression to remove special characters
    """
    if type(words) is str:
        words = [words]
    sep_re = r'[\s\(\){}\[\];,\.]+'
    num_re = r'\d'
    words = re.split(sep_re, ' '.join(words).lower())
    words = [w for w in words if len(w) >= 3 and not re.match(num_re, w)]
    if unique:
        words = list(set(words))
        words = set(__normalize_words(words))
    else:
        words = __normalize_words(words)
    return words

def links_to_word(text: str) -> str:
    return re.sub("https?:\/\/[^\s]+", " link ", text)

def no_char(text: str) -> str:
    text = re.sub(r"\s+[a-zA-Z]\s+", " ", text)
    text = re.sub(r"\^[a-zA-Z]\s+", " ", text)
    text = re.sub(r"\s+[a-zA-Z]$", " ", text)
    return text

def no_html_tags(text: str) -> str:
    return re.sub("<.*?>", " ", text)

def no_multi_spaces(text: str) -> str:
    return re.sub(r"\s+", " ", text, flags=re.I)

def lemmatize(text: str) -> str:
    tokens = text.split()
    tokens = [stemmer.lemmatize(word) for word in tokens]
    return " ".join(tokens)

def underscore_to_space(text: str) -> str:
    text = text.replace("_", " ")
    text = text.replace("-", " ")
    return text

def preprocess_text(source: str) -> str:
    # Remove all the special characters
    source = re.sub(r'\W', ' ', str(source))
    source = re.sub(r'^b\s+', '', source)
    source = source.lower()
    return source

def cleaning_words(text: str) -> str:
    tmp_text = links_to_word(text)
    tmp_text = no_html_tags(tmp_text)
    tmp_text = underscore_to_space(tmp_text)
    tmp_text = no_char(tmp_text)
    tmp_text = preprocess_text(tmp_text)
    tmp_text = no_multi_spaces(tmp_text)
    return tmp_text

In [28]:
""" Test for Text Cleaning """
test_source = full_df.source.to_numpy()

for i in range(len(test_source[0])):
    print(cleaning_words(test_source[i]))

 this python 3 environment comes with many helpful analytics libraries installed it is defined by the kaggle python docker image link for example here s several helpful packages to load import numpy as np linear algebra import pandas as pd data processing csv file i o e g pd read csv input data files are available in the read only input directory for example running this by clicking run or pressing shift enter will list all files under the input directory import os for dirname filenames in os walk kaggle input for filename in filenames print os path join dirname filename you can write up to 20gb to the current directory kaggle working that gets preserved as output when you create version using save run all you can also write temporary files to kaggle temp but they won t be saved outside of the current session
import numpy as np import pandas as pd import random from sklearn model selection import train test split cross val score from sklearn preprocessing import standardscaler robustsc